# Assignment 1, 8-Puzzle Programming

Your homework must be implemented in this Notebook file. 
You can add as many cells as you want. However, you are not allowed to touch the code below the line "=============".
You need to implement the three (four for grads) searching functions and the print result functions.
For the searching functions, feel free to customize the return data types and parameter lists as long as the function name is as required.


In [ ]:
#implementation of function "Iterative_deepening_DFS"
from numpy import *
import numpy as np
import queue 
import gc
import time

GOAL_STATE = [0, 1, 2, 3, 4, 5, 6, 7, 8]
parent=0

def swap(state,index1,index2):
    temp=state[index1]
    state[index1]=state[index2]
    state[index2]=temp
    return state

def move_state(state,move):
    next_state=list(state)
    index=state.index(0)
   
    if move =='R':
        if index % 3 == 2:
            return None
        next_state = swap(next_state,index,index+1)
    elif move=='L':
        if index%3 ==0:
            return None
        next_state=swap(next_state,index,index-1)
    elif move == 'U':
        if index<3:
            return None
        next_state=swap(next_state,index,index-3)
    elif move == 'D':
        if index > 5:
            return None
        next_state=swap(next_state,index,index + 3)
    else:
        return None
    return next_state

def move(state):
    next_state=[]
    global parent
    ty_state = move_state(state,'R')
    if ty_state is not None:
        next_state.append(ty_state)
        
    ty_state =move_state(state,'L')
    if ty_state is not None:
        next_state.append(ty_state)
        
    ty_state =move_state(state,'U')
    if ty_state is not None:
        next_state.append(ty_state)
        
    ty_state =move_state(state,'D')
    if ty_state is not None:
        next_state.append(ty_state)

    
    
    #print("ceng",next_state)
    return next_state 
    
        
            
def search(list_input):
    puzzle=list()
    puzzle_cp=set()
    puzzle_cp.add(tuple(list_input))
    
    puzzle.append(list_input)
    explored = set()
    
    
    #puzzle_cp.add(list_input)
    t=0
    while puzzle:
        #print("puzzle:", puzzle)
        c_state = puzzle.pop()
        
        puzzle_cp.remove(tuple(c_state))
        explored.add(tuple(c_state))
        if c_state == GOAL_STATE:
            print("step",len(explored))
            #free memory
            del puzzle
            del explored
            del puzzle_cp
            return c_state
        # reversed because avoid BUG
        #some reason:http://www.cnblogs.com/bananaplan/p/remove-listitem-while-iterating.html
        for move_state in reversed(move(c_state)):
            
            if  tuple(move_state) not in explored and tuple(move_state) not in puzzle_cp :
                #print(move_state)
              
                puzzle.append(move_state)
                puzzle_cp.add(tuple(move_state))
                
        
    return None
              
with open('Input8PuzzleCasesUpdates.txt') as f:
    listRow = f.readlines();
    average=0
    for i in range(100):  
        start_time=time.clock()
        #string to list
        list_input = listRow[i].split(",")
        list_input = [int(x) for x in list_input]
        print("print list:",list_input)
        
        #state=search([4, 3, 1, 0, 2, 6, 7, 5, 8])
        state=search(list_input)
        while state is None:
            gc.collect()
            print("false")
            break
            #state=search(list_input)

                
        
        print("print list successful:",i,":",state)
        endtime= time.clock()-start_time
        print("time:", endtime)
        average=average+endtime
        print("================================================")
        gc.collect()
        #listinput = np.matrix(listRow[i])
        #a= listinput.reshape((3,3));
        #
        #print(a)
         
    
    print("Average time:",average/100)

print list: [8, 7, 5, 4, 1, 2, 3, 0, 6]
step 125210
print list successful: 0 : [0, 1, 2, 3, 4, 5, 6, 7, 8]
time: 0.9627879999999891
print list: [7, 8, 6, 0, 2, 5, 1, 4, 3]
step 125195
print list successful: 1 : [0, 1, 2, 3, 4, 5, 6, 7, 8]
time: 0.9270170000000064
print list: [1, 0, 7, 5, 3, 6, 8, 4, 2]
step 148303
print list successful: 2 : [0, 1, 2, 3, 4, 5, 6, 7, 8]
time: 1.0991699999999867
print list: [3, 8, 4, 0, 1, 7, 5, 2, 6]
step 121851
print list successful: 3 : [0, 1, 2, 3, 4, 5, 6, 7, 8]
time: 0.9416400000000067
print list: [6, 1, 2, 7, 0, 8, 5, 4, 3]
step 88514
print list successful: 4 : [0, 1, 2, 3, 4, 5, 6, 7, 8]
time: 0.7058800000000076
print list: [2, 5, 3, 1, 4, 6, 8, 0, 7]
step 10026
print list successful: 5 : [0, 1, 2, 3, 4, 5, 6, 7, 8]
time: 0.079654000000005
print list: [6, 8, 5, 3, 4, 7, 2, 1, 0]
step 165160
print list successful: 6 : [0, 1, 2, 3, 4, 5, 6, 7, 8]
time: 1.2257579999999848
print list: [2, 7, 8, 0, 1, 5, 4, 3, 6]
step 41865
print list successful: 7 : [

In [ ]:
#implementation of function "aStarMisplacedTiles"


In [ ]:
#implementation of function "aStarManhattanDistance"


In [ ]:
#implementation of function "breadthFirstSearch" (for graduate students)


In [ ]:
#implementation of function "print_result(result)"

You can insert as many cells as you want above
You are not Allowed to modify the code below this line.
===============================

In [ ]:
#you need to implement print_result function to print out the result according to the required format
print_result(result)


The output format should be as follows. You only need to give one sample solution as an example.
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Solution of the first Scenario:
X X X
X X X
X X X
to
X X X
X X X
X X X
to
X X X
X X X
X X X
to
X X X
X X X
X X X
to
.
.
.
0 1 2
3 4 5
6 7 8

                Average_Steps    Average_Time   Average_Iterations   
UCS
A*(Misplaced)
A*(Manhattan)
BFS

